In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
np.random.seed(42)

In [2]:
df = pd.read_csv(r"train.csv")
dfTest = pd.read_csv(r"test.csv")

In [6]:
dfTest.drop(columns=['Name'], inplace=True)
dfTest.drop(columns=['PassengerId'], inplace=True)

df.drop(columns=['Name'], inplace=True)
df.drop(columns=['PassengerId'], inplace=True)

newTic=np.empty(0)
for i in range(0,891):
    spT=df.iloc[i]['Ticket'].split( )
    if(len(spT)==1):
        newTic=np.append(newTic,["Regular"])
        #print(df.iloc[i]['Ticket'].split( )," - ")
    else:
        newTic=np.append(newTic,spT[0])
        #print(df.iloc[i]['Ticket'].split( ),spT[0])
pd.set_option('display.max_rows', None)
df['Ticket']=pd.Series(newTic)
newTic=np.empty(0)
for i in range(0,418):
    spT=dfTest.iloc[i]['Ticket'].split( )
    if(len(spT)==1):
        newTic=np.append(newTic,["Regular"])
        #print(df.iloc[i]['Ticket'].split( )," - ")
    else:
        newTic=np.append(newTic,spT[0])
        #print(df.iloc[i]['Ticket'].split( ),spT[0])

dfTest['Ticket']=pd.Series(newTic)

In [7]:
one= pd.concat([df[df['Pclass'] == 1]['Age'],dfTest[dfTest['Pclass'] == 1]['Age']], ignore_index=True)
two= pd.concat([df[df['Pclass'] == 2]['Age'],dfTest[dfTest['Pclass'] == 2]['Age']], ignore_index=True)
there= pd.concat([df[df['Pclass'] == 2]['Age'],dfTest[dfTest['Pclass'] == 2]['Age']], ignore_index=True)
Pcl=[0,one.median(),two.median(),there.median()]
j=0;
for label,content in df.items():
    if (label== 'Age'):
        for i in range(0,891):
            if(np.isnan(df.iloc[i][label])):
                P=df.iloc[i]['Pclass']
                df.loc[df.index[i],'Age']=Pcl[P]

for label,content in dfTest.items():
    if (label== 'Age'):
        for i in range(0,418):
            if(np.isnan(dfTest.iloc[i][label])):
                P=dfTest.iloc[i]['Pclass']
                dfTest.loc[dfTest.index[i],'Age']=Pcl[P]
df.loc[61,'Embarked']='S'
df.loc[829,'Embarked']='S'

In [8]:
arr1=df.loc[df['Pclass'] == 1 ,'Cabin'].unique()
arr2=df.loc[df['Pclass'] == 2 ,'Cabin'].unique()
arr3=df.loc[df['Pclass'] == 3 ,'Cabin'].unique()
print(len(arr1),len(arr2),len(arr3))
arr1=np.append(arr1,dfTest.loc[dfTest['Pclass'] == 1 ,'Cabin'].unique())
arr2=np.append(arr2,dfTest.loc[dfTest['Pclass'] == 2 ,'Cabin'].unique())
arr3=np.append(arr3,dfTest.loc[dfTest['Pclass'] == 3 ,'Cabin'].unique())
print(len(arr1),len(arr2),len(arr3))
firstClass=[]
scClass=[]
thClass=[]
for i in arr1:
    if(type(i)==str):
        #print(i,i[0])
        firstClass.append(i[0])
for i in arr2:
    if(type(i)==str):
        #print(i,i[0])
        scClass.append(i[0])
for i in arr3:
    if(type(i)==str):
        #print(i,i[0])
        thClass.append(i[0])

134 8 8
201 15 13


In [9]:
print(set(firstClass))
print(set(scClass))
print(set(thClass))

{'C', 'E', 'D', 'A', 'B', 'T'}
{'E', 'D', 'F'}
{'E', 'F', 'G'}


In [10]:
cb=["x","1","2","3"]
for i in range(0,891):
    #print(df.loc[df.index == i, 'Cabin'],df.iloc[i]['Cabin'])
    if (type(df.loc[df.index[i],'Cabin']) == str):
       #df.loc[df.index[i],'Cabin']=df.loc[df.index[i],'Cabin'][0]+cb[df.loc[df.index[i],'Pclass']]
        df.loc[df.index[i],'Cabin']=df.loc[df.index[i],'Cabin'][0]
    else:
        df.loc[df.index[i],'Cabin']='M'+cb[df.loc[df.index[i],'Pclass']]

for i in range(0,418):
    #print(df.loc[df.index == i, 'Cabin'],df.iloc[i]['Cabin'])
    if (type(dfTest.loc[df.index[i],'Cabin']) == str):
       #dfTest.loc[dfTest.index[i],'Cabin']=dfTest.loc[dfTest.index[i],'Cabin'][0]+cb[dfTest.loc[dfTest.index[i],'Pclass']]
        dfTest.loc[dfTest.index[i],'Cabin']=dfTest.loc[dfTest.index[i],'Cabin'][0]
    else:
        dfTest.loc[dfTest.index[i],'Cabin']='M'+cb[dfTest.loc[dfTest.index[i],'Pclass']]
print(df.isnull().sum(),dfTest.isnull().sum())

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Cabin       0
Embarked    0
dtype: int64 Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        1
Cabin       0
Embarked    0
dtype: int64


In [11]:
dfTest.loc[dfTest.index[152],'Fare']=9000
print(df.isnull().sum(),dfTest.isnull().sum())

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Cabin       0
Embarked    0
dtype: int64 Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Cabin       0
Embarked    0
dtype: int64


In [13]:
for label,content in df.items():
    if pd.api.types.is_object_dtype(content):
        print(label)
        df[label] = content.astype("category").cat.as_ordered()
for label,content in dfTest.items():
    if pd.api.types.is_object_dtype(content):
        print(label)
        dfTest[label] = content.astype("category").cat.as_ordered()

Sex
Ticket
Cabin
Embarked
Sex
Ticket
Cabin
Embarked


In [15]:
for label, content in df.items():
    if not pd.api.types.is_numeric_dtype(content):
        df[label] = pd.Categorical(content).codes
for label, content in dfTest.items():
    if not pd.api.types.is_numeric_dtype(content):
        dfTest[label] = pd.Categorical(content).codes  

In [26]:
np.random.seed(42)

X = df.drop("Survived", axis =1)
y = df.Survived.values

X_train,X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2,random_state=42)

In [27]:
modelReg= LogisticRegression(solver="liblinear",random_state=42)
modelReg.fit(X_train, y_train)
modelReg.score(X_valid,y_valid)

0.7988826815642458

In [28]:
modelRF= RandomForestClassifier(random_state=42)
modelRF.fit(X_train, y_train)
modelRF.score(X_valid, y_valid)

0.8156424581005587

In [25]:
log_reg_grid = {"C": np.logspace(-4, 4, 20),
                "solver": ["liblinear"]}

# Different RandomForestClassifier hyperparameters
rf_grid = {"n_estimators": np.arange(10, 1100, 50),
           "max_depth": [None, 3, 5, 10],
           "random_state": [42]
          }

"""rs_rf= RandomizedSearchCV(RandomForestClassifier(),
                          param_distributions=rf_grid,
                          cv=5,
                          n_iter=100,
                          verbose=True)"""
gr_rf= GridSearchCV(RandomForestClassifier(),
                    param_grid=rf_grid,
                    cv=5,
                    verbose=True)


reg= GridSearchCV(LogisticRegression(),
                    param_grid=log_reg_grid,
                    cv=5,
                    verbose=True)

gr_rf.fit(X_train, y_train)
gr_rf.score(X_valid, y_valid)

Fitting 5 folds for each of 88 candidates, totalling 440 fits


0.7988826815642458

In [29]:
dfTest

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,1,34.50,0,0,17,7.8292,9,1
1,3,0,47.00,1,0,17,7.0000,9,2
2,2,1,62.00,0,0,17,9.6875,8,1
3,3,1,27.00,0,0,17,8.6625,9,2
4,3,0,22.00,1,1,17,12.2875,9,2
5,3,1,14.00,0,0,17,9.2250,9,2
6,3,0,30.00,0,0,17,7.6292,9,1
7,2,1,26.00,1,1,17,29.0000,8,2
8,3,0,18.00,0,0,17,7.2292,9,0
9,3,1,21.00,2,0,3,24.1500,9,2
